In [3]:

import torch
torch.cuda.is_available()



True

In [4]:
# For Data

from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient

#from Data.historical_data import Historical_data
import pandas as pd
from Preprocessing.preprocessing import preprocessing_improved
from Models.LSTM.lightningLSTM import LightningLSTM
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from Preprocessing.dataclasses import StockPriceDataModule, StockDataset
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
import matplotlib.pyplot as plt
import numpy as np 


In [5]:
new_data = False

if new_data:

    stocks = [['AAPL'], ['MSFT'], ['GOOG'], ['TSLA'], ['NVDA']]

    for stock in stocks:

        timeframes = [TimeFrame.Day, TimeFrame.Hour, TimeFrame.Minute]
        time_strings = ['Day', 'Hour', 'Minute']

        for timeframe, time_string in zip(timeframes, time_strings):

            s_type = "StockBars"
            stock = stock
            timeframe = timeframe
            start = "2020-08-20 06:00:00"
            end = "2021-08-20 06:00:00"
            client = StockHistoricalDataClient("PKV7BABG3DGE0GYL137W", "af1eQghpVgbyX2lh2T9n2ColmVYXHCWndQbKR3Lr")
            save_csv = True
            time_string = time_string

            df = Historical_data(s_type = s_type, stock = stock, timeframe = timeframe, start = start, end = end, client = client, save_csv = save_csv , time_string = time_string)

else:

    df = pd.read_csv('Data\Stock\StockBars\MSFT_Minute')

In [6]:
data_train, data_test = preprocessing_improved(df = df[:int(len(df)*0.1)], lag = 1, sequence_length=100, dif_all=True, train_size=0.9 )
data_module = StockPriceDataModule(train_sequence = data_train,  test_sequence= data_test, batch_size=64, num_workers = 4)
data_module.setup()

for item in data_module.train_dataloader():
    print(item.keys())
    print(item['sequence'].shape)
    print(item['label'].shape)
    break


dict_keys(['sequence', 'label'])
torch.Size([64, 100, 8])
torch.Size([64])


In [16]:
learning_model = LightningLSTM(input_size=8, hidden_size = 100, num_layers = 1)
trainer = pl.Trainer(auto_lr_find='learning_rate')
trainer.tune(learning_model)
learning_model.learning_rate

TypeError: __init__() missing 1 required positional argument: 'learning_rate'

In [5]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best_checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = TensorBoardLogger("logs/", name="lightninglstm")

early_stopping_callback = EarlyStopping(monitor="val_loss", patience=2)

progressbar = TQDMProgressBar(refresh_rate=100)

In [8]:
model = LightningLSTM(input_size=8, hidden_size = 100, num_layers = 1)
trainer = pl.Trainer(max_epochs=5, callbacks=[checkpoint_callback, early_stopping_callback,progressbar], accelerator="gpu", devices=1)
trainer.fit(model, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | model     | LSTM    | 44.1 K
1 | criterion | MSELoss | 0     
--------------------------------------
44.1 K    Trainable params
0         Non-trainable params
44.1 K    Total params
0.176     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 219: 'val_loss' reached 1.16291 (best 1.16291), saving model to 'C:\\Users\\lukas\\Programmering\\Investment AI\\Master-Thesis\\checkpoints\\best_checkpoint-v1.ckpt' as top 1


...

Epoch 2, global step 657: 'val_loss' reached 0.76936 (best 0.76936), saving model to 'C:\\Users\\lukas\\Programmering\\Investment AI\\Master-Thesis\\checkpoints\\best_checkpoint-v1.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Epoch 3, global step 876: 'val_loss' reached 0.73478 (best 0.73478), saving model to 'C:\\Users\\lukas\\Programmering\\Investment AI\\Master-Thesis\\checkpoints\\best_checkpoint-v1.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Epoch 4, global step 1095: 'val_loss' reached 0.71279 (best 0.71279), saving model to 'C:\\Users\\lukas\\Programmering\\Investment AI\\Master-Thesis\\checkpoints\\best_checkpoint-v1.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=5` reached.


In [9]:

model.freeze()

test = StockDataset(data_test)

prede = []
labels = []

for item in test:
  sequence = item["sequence"]
  label = item['label']

  _, output = model(sequence.unsqueeze(dim=0))
  prede.append(float(output.numpy()[0]))
  labels.append(float(label.numpy()))

In [10]:
trained_model = LightningLSTM.load_from_checkpoint("checkpoints\\best_checkpoint.ckpt",input_size=8, hidden_size = 100, num_layers = 1)
trained_model.freeze()

test = StockDataset(data_test)

prede = []
labels = []

for item in test:
  sequence = item["sequence"]
  label = item['label']

  _, output = trained_model(sequence.unsqueeze(dim=0))
  prede.append(float(output.numpy()[0]))
  labels.append(float(label.numpy()))

C:\Users\lukas\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\utilities\migration\utils.py:49: PossibleUserWarning: The loaded checkpoint was produced with Lightning v1.9.2, which is newer than your current Lightning version: v1.9.1
  rank_zero_warn(


In [11]:
count = np.count_nonzero(np.array(prede) * np.array(labels) > 0)

accuracy = count / len(prede)

# Create a plot of the last 100 values of each time series
plt.plot(prede[-100:], label='prede')
plt.plot(labels[-100:], label='labels')
plt.title(accuracy)
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'